In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['giba-r-data-table-simple-features-1-17-lb_train.pkl',
 'eem_train.pkl',
 'coulomb-interaction-speed-up_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'angles-and-distances_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'coulomb-interaction-speed-up_test.pkl',
 'bonds-from-structure-data_train.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'eem_test.pkl',
 'brute-force-feature-engineering_train.pkl',
 'molecule-with-openbabel_train.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'eachtype_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'bonds-from-structure-data_test.pkl',
 'molecule-with-openbabel_test.pkl',
 'eachtype_te

In [ ]:
y=fc, type=0
227
{'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'tertiary_atom_1', 'adC2', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'atom_index_farthest_0', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'dist_H_3_x', 'tertiary_angle_3', 'dist_to_type_std', 'dist_C_3_y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'linkM0', 'dist_H_2_y', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_O_1_y', 'dist_C_2_x', 'eem_0', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'distance_farthest_0', 'yukawa_N.x', 'dist_H_3_y', 'molecule_type_dist_mean_diff', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'dist_C_0_y', 'dist_H_2_x', 'dist_to_type_0_mean', 'tertiary_distance_5', 'atom_1_n_bonds', 'dist_C_3_x', 'tertiary_distance_0', 'dist_to_type_mean', 'distC1', 'dist_O_0_y', 'dist_C_4_y', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'yukawa_H.x', 'tertiary_atom_0', 'tertiary_angle_4', 'inv_dist1E', 'tertiary_angle_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=1
561
{'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'linkM1', 'adC2', 'tertiary_distance_1', 'tertiary_distance_6', 'dist_C_2_y', 'distance_center0', 'dist_C_1_y', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'yukawa_C.x', 'inv_dist1', 'cos_center0', 'tertiary_angle_3', 'cos_center1', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'inv_distP', 'vander_N.y', 'dist_H_2_y', 'dist_H_0_x', 'tertiary_angle_0', 'vander_N.x', 'dist_C_2_x', 'eem_0', 'coulomb_H.x', 'min_molecule_atom_1_dist_xyz', 'yukawa_N.y', 'molecule_atom_index_0_dist_mean_div', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'linkN', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'distance_center1', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'link0', 'yukawa_N.x', 'dist_H_3_y', 'dist_N_0_x', 'dist_C_0_y', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'inv_distPE', 'vander_H.x', 'tertiary_distance_5', 'dist_N_1_y', 'dist_N_1_x', 'dist_C_3_x', 'tertiary_distance_0', 'dist_O_0_y', 'distN0', 'dist_C_4_y', 'cos_f1', 'tertiary_distance_2', 'vander_O.x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_H_1_y', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'tertiary_atom_0', 'distN1', 'tertiary_angle_1', 'adC4', 'bond_atom'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=2
788
{'columns': ['atom_index_1_cycle_size_mean', 'atom_0_bond_lengths_max', 'dist_H_0_y', 'tertiary_atom_1', 'adC2', 'tertiary_atom_2', 'tertiary_distance_6', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'dist_C_0_x', 'inv_dist1R', 'atom_1_bond_lengths_min', 'cos_c0_c1', 'tertiary_angle_3', 'inv_dist1', 'molecule_atom_index_0_dist_max_diff', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_C_2_x', 'molecule_atom_index_0_dist_mean_div', 'molecule_atom_index_0_dist_min_diff', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'inv_dist0R', 'distance_farthest_0', 'dist_N_0_x', 'dist_C_0_y', 'atom_index_1_n_cycle', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'cos_c0', 'distC0', 'tertiary_atom_3', 'bond_atom', 'molecule_atom_index_1_dist_min_div', 'atom_1_n_bonds', 'vander_C.y', 'dist_C_3_x', 'tertiary_atom_4', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'atom_1_bond_lengths_mean', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'tertiary_atom_0', 'tertiary_angle_4', 'tertiary_angle_1', 'cos_c1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=3
294
{'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=4
468
{'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=5
179
{'columns': ['tertiary_distance_6', 'dist_C_4_y', 'molecule_atom_index_0_dist_std_diff', 'inv_dist1R', 'molecule_atom_index_1_dist_min_div', 'tertiary_distance_3', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'tertiary_atom_2', 'vander_O.x', 'atom_1_bond_lengths_max', 'dist_O_0_y', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_1_dist_min_div', 'adC1', 'adC2', 'molecule_atom_index_1_dist_min_diff', 'adN1', 'dist_N_0_y', 'atom_index_1_n_cycle', 'inv_dist1E', 'molecule_type_0_dist_std_diff', 'inv_dist0', 'molecule_atom_index_0_dist_min_diff', 'mean_molecule_atom_1_dist_xyz', 'atom_index_1_cycle_size_mean', 'dist_C_0_y', 'dist_xyz', 'tertiary_angle_4', 'dist_H_1_x', 'atom_0_bond_lengths_max', 'molecule_type_dist_mean_div', 'tertiary_angle_3', 'cos_c1', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'dist_C_3_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_1', 'molecule_atom_index_1_dist_std_diff', 'dist_C_0_x', 'dist_N_0_x', 'dist_O_1_y', 'dist_H_0_x', 'cos_f0', 'tertiary_atom_1', 'vander_O.y', 'tertiary_angle_2', 'eem_0', 'dist_C_2_y', 'adC3', 'molecule_atom_1_dist_min_diff', 'tertiary_distance_1', 'dist_O_1_x', 'dist_to_type_mean', 'tertiary_distance_5', 'dist_C_1_x', 'dist_H_1_y', 'dist_H_0_y', 'molecule_atom_index_0_dist_min_div', 'dist_H_2_y', 'tertiary_distance_2', 'molecule_type_dist_std_diff', 'eem_1', 'dist_C_2_x', 'distC0', 'molecule_atom_index_0_dist_max_diff', 'inv_dist1', 'dist_C_1_y', 'tertiary_angle_0', 'dist_O_0_x', 'link0', 'mean_molecule_atom_0_dist_xyz', 'dist_C_3_y', 'tertiary_atom_3', 'cos_c0_c1', 'cos_c0', 'inv_dist0R'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=6
269
{'columns': ['cos_c0_c1', 'tertiary_angle_0', 'cos_c0', 'cos_c1', 'max_molecule_atom_1_dist_xyz', 'molecule_atom_index_0_dist_min_diff', 'dist_H_0_y', 'dist_O_0_y', 'dist_O_0_x', 'dist_C_2_y', 'dist_C_1_y', 'tertiary_angle_1', 'dist_C_0_y', 'molecule_atom_index_1_dist_max_div', 'tertiary_atom_1', 'tertiary_distance_1', 'dist_C_0_x', 'tertiary_distance_3', 'adC3', 'molecule_atom_index_0_dist_min_div', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'inv_distPR', 'tertiary_distance_2', 'atom_index_closest_1', 'dist_N_0_y', 'tertiary_angle_2', 'dist_to_type_mean', 'dist_C_2_x', 'distance_c1', 'dist_H_1_y', 'dist_H_0_x', 'cos_f0', 'dist_C_1_x', 'adC2', 'eem_1', 'dist_to_type_0_mean', 'dist_to_type_std', 'max_distance_y', 'distH0', 'atom_1_bond_lengths_mean', 'cos_f1', 'tertiary_atom_0', 'dist_C_3_y', 'dist_xyz', 'dist_to_type_1_mean', 'dist_C_3_x', 'inv_dist0R', 'adN1', 'tertiary_atom_2', 'molecule_atom_index_1_dist_max_diff', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'dist_H_2_y', 'dist_H_1_x', 'linkN', 'cos_f0_f1', 'adC1', 'adH1', 'tertiary_angle_4', 'adH2', 'molecule_type_dist_std_diff', 'yukawa_C.y', 'dist_O_1_x', 'linkM0', 'molecule_atom_index_0_dist_max_div', 'link0', 'molecule_atom_index_0_dist_max_diff', 'dist_H_2_x', 'dist_C_4_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=7
883
{'columns': ['tertiary_angle_1', 'dist_C_4_y', 'molecule_atom_index_0_dist_std_diff', 'inv_dist1R', 'dist_C_0_x', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_1_dist_max_div', 'atom_1_n_bonds', 'dist_H_0_x', 'tertiary_atom_2', 'cos_f0', 'tertiary_atom_1', 'atom_1_bond_lengths_max', 'dist_O_0_y', 'dist_to_type_std', 'vander_O.y', 'tertiary_angle_2', 'yukawa_H.y', 'dist_C_2_y', 'adC3', 'adC1', 'tertiary_distance_1', 'adC2', 'dist_H_0_y', 'dist_C_1_x', 'molecule_atom_index_1_dist_min_diff', 'dist_H_1_y', 'bond_atom', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'dist_H_2_y', 'dist_N_0_y', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_diff', 'eem_1', 'atom_index_1_cycle_size_mean', 'dist_C_0_y', 'dist_C_2_x', 'dist_xyz', 'dist_H_1_x', 'dist_C_4_x', 'molecule_atom_index_0_dist_std_div', 'dist_C_1_y', 'tertiary_angle_0', 'dist_O_0_x', 'dist_to_type_1_mean', 'inv_distPR', 'atom_1_bond_lengths_min', 'min_molecule_atom_0_dist_xyz', 'dist_C_3_y', 'tertiary_angle_3', 'cos_c1', 'tertiary_distance_4', 'tertiary_atom_3', 'atom_1_bond_lengths_mean', 'dist_C_3_x', 'dist_H_3_y', 'cos_c0_c1', 'cos_c0', 'inv_dist0R', 'max_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}

In [5]:
mytrial = []

In [6]:
# ''scalar_coupling_constant, 'fc', 'sd','pso','dso'
for y in ['fc']:
    for t in np.arange(5,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=fc, type=5
179
{'columns': ['tertiary_distance_6', 'dist_C_4_y', 'molecule_atom_index_0_dist_std_diff', 'inv_dist1R', 'molecule_atom_index_1_dist_min_div', 'tertiary_distance_3', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'tertiary_atom_2', 'vander_O.x', 'atom_1_bond_lengths_max', 'dist_O_0_y', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_1_dist_min_div', 'adC1', 'adC2', 'molecule_atom_index_1_dist_min_diff', 'adN1', 'dist_N_0_y', 'atom_index_1_n_cycle', 'inv_dist1E', 'molecule_type_0_dist_std_diff', 'inv_dist0', 'molecule_atom_index_0_dist_min_diff', 'mean_molecule_atom_1_dist_xyz', 'atom_index_1_cycle_size_mean', 'dist_C_0_y', 'dist_xyz', 'tertiary_angle_4', 'dist_H_1_x', 'atom_0_bond_lengths_max', 'molecule_type_dist_mean_div', 'tertiary_angle_3', 'cos_c1', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'dist_C_3_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_1', 'molecule_atom_index_1_dist_std_diff', 'dist_C_0_x', 'dist_N_0_x', 'dist_O_1_y', 'di

In [7]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['fc']

In [8]:
columns_list = [{'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'tertiary_atom_1', 'adC2', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'atom_index_farthest_0', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'dist_H_3_x', 'tertiary_angle_3', 'dist_to_type_std', 'dist_C_3_y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'linkM0', 'dist_H_2_y', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_O_1_y', 'dist_C_2_x', 'eem_0', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'distance_farthest_0', 'yukawa_N.x', 'dist_H_3_y', 'molecule_type_dist_mean_diff', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'dist_C_0_y', 'dist_H_2_x', 'dist_to_type_0_mean', 'tertiary_distance_5', 'atom_1_n_bonds', 'dist_C_3_x', 'tertiary_distance_0', 'dist_to_type_mean', 'distC1', 'dist_O_0_y', 'dist_C_4_y', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'yukawa_H.x', 'tertiary_atom_0', 'tertiary_angle_4', 'inv_dist1E', 'tertiary_angle_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'linkM1', 'adC2', 'tertiary_distance_1', 'tertiary_distance_6', 'dist_C_2_y', 'distance_center0', 'dist_C_1_y', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'yukawa_C.x', 'inv_dist1', 'cos_center0', 'tertiary_angle_3', 'cos_center1', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'inv_distP', 'vander_N.y', 'dist_H_2_y', 'dist_H_0_x', 'tertiary_angle_0', 'vander_N.x', 'dist_C_2_x', 'eem_0', 'coulomb_H.x', 'min_molecule_atom_1_dist_xyz', 'yukawa_N.y', 'molecule_atom_index_0_dist_mean_div', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'linkN', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'distance_center1', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'link0', 'yukawa_N.x', 'dist_H_3_y', 'dist_N_0_x', 'dist_C_0_y', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'inv_distPE', 'vander_H.x', 'tertiary_distance_5', 'dist_N_1_y', 'dist_N_1_x', 'dist_C_3_x', 'tertiary_distance_0', 'dist_O_0_y', 'distN0', 'dist_C_4_y', 'cos_f1', 'tertiary_distance_2', 'vander_O.x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_H_1_y', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'tertiary_atom_0', 'distN1', 'tertiary_angle_1', 'adC4', 'bond_atom'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['atom_index_1_cycle_size_mean', 'atom_0_bond_lengths_max', 'dist_H_0_y', 'tertiary_atom_1', 'adC2', 'tertiary_atom_2', 'tertiary_distance_6', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'dist_C_0_x', 'inv_dist1R', 'atom_1_bond_lengths_min', 'cos_c0_c1', 'tertiary_angle_3', 'inv_dist1', 'molecule_atom_index_0_dist_max_diff', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_C_2_x', 'molecule_atom_index_0_dist_mean_div', 'molecule_atom_index_0_dist_min_diff', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'inv_dist0R', 'distance_farthest_0', 'dist_N_0_x', 'dist_C_0_y', 'atom_index_1_n_cycle', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'cos_c0', 'distC0', 'tertiary_atom_3', 'bond_atom', 'molecule_atom_index_1_dist_min_div', 'atom_1_n_bonds', 'vander_C.y', 'dist_C_3_x', 'tertiary_atom_4', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'atom_1_bond_lengths_mean', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'tertiary_atom_0', 'tertiary_angle_4', 'tertiary_angle_1', 'cos_c1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['tertiary_distance_6', 'dist_C_4_y', 'molecule_atom_index_0_dist_std_diff', 'inv_dist1R', 'molecule_atom_index_1_dist_min_div', 'tertiary_distance_3', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'tertiary_atom_2', 'vander_O.x', 'atom_1_bond_lengths_max', 'dist_O_0_y', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_1_dist_min_div', 'adC1', 'adC2', 'molecule_atom_index_1_dist_min_diff', 'adN1', 'dist_N_0_y', 'atom_index_1_n_cycle', 'inv_dist1E', 'molecule_type_0_dist_std_diff', 'inv_dist0', 'molecule_atom_index_0_dist_min_diff', 'mean_molecule_atom_1_dist_xyz', 'atom_index_1_cycle_size_mean', 'dist_C_0_y', 'dist_xyz', 'tertiary_angle_4', 'dist_H_1_x', 'atom_0_bond_lengths_max', 'molecule_type_dist_mean_div', 'tertiary_angle_3', 'cos_c1', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'dist_C_3_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_1', 'molecule_atom_index_1_dist_std_diff', 'dist_C_0_x', 'dist_N_0_x', 'dist_O_1_y', 'dist_H_0_x', 'cos_f0', 'tertiary_atom_1', 'vander_O.y', 'tertiary_angle_2', 'eem_0', 'dist_C_2_y', 'adC3', 'molecule_atom_1_dist_min_diff', 'tertiary_distance_1', 'dist_O_1_x', 'dist_to_type_mean', 'tertiary_distance_5', 'dist_C_1_x', 'dist_H_1_y', 'dist_H_0_y', 'molecule_atom_index_0_dist_min_div', 'dist_H_2_y', 'tertiary_distance_2', 'molecule_type_dist_std_diff', 'eem_1', 'dist_C_2_x', 'distC0', 'molecule_atom_index_0_dist_max_diff', 'inv_dist1', 'dist_C_1_y', 'tertiary_angle_0', 'dist_O_0_x', 'link0', 'mean_molecule_atom_0_dist_xyz', 'dist_C_3_y', 'tertiary_atom_3', 'cos_c0_c1', 'cos_c0', 'inv_dist0R'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['cos_c0_c1', 'tertiary_angle_0', 'cos_c0', 'cos_c1', 'max_molecule_atom_1_dist_xyz', 'molecule_atom_index_0_dist_min_diff', 'dist_H_0_y', 'dist_O_0_y', 'dist_O_0_x', 'dist_C_2_y', 'dist_C_1_y', 'tertiary_angle_1', 'dist_C_0_y', 'molecule_atom_index_1_dist_max_div', 'tertiary_atom_1', 'tertiary_distance_1', 'dist_C_0_x', 'tertiary_distance_3', 'adC3', 'molecule_atom_index_0_dist_min_div', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'inv_distPR', 'tertiary_distance_2', 'atom_index_closest_1', 'dist_N_0_y', 'tertiary_angle_2', 'dist_to_type_mean', 'dist_C_2_x', 'distance_c1', 'dist_H_1_y', 'dist_H_0_x', 'cos_f0', 'dist_C_1_x', 'adC2', 'eem_1', 'dist_to_type_0_mean', 'dist_to_type_std', 'max_distance_y', 'distH0', 'atom_1_bond_lengths_mean', 'cos_f1', 'tertiary_atom_0', 'dist_C_3_y', 'dist_xyz', 'dist_to_type_1_mean', 'dist_C_3_x', 'inv_dist0R', 'adN1', 'tertiary_atom_2', 'molecule_atom_index_1_dist_max_diff', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'dist_H_2_y', 'dist_H_1_x', 'linkN', 'cos_f0_f1', 'adC1', 'adH1', 'tertiary_angle_4', 'adH2', 'molecule_type_dist_std_diff', 'yukawa_C.y', 'dist_O_1_x', 'linkM0', 'molecule_atom_index_0_dist_max_div', 'link0', 'molecule_atom_index_0_dist_max_diff', 'dist_H_2_x', 'dist_C_4_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['tertiary_angle_1', 'dist_C_4_y', 'molecule_atom_index_0_dist_std_diff', 'inv_dist1R', 'dist_C_0_x', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_1_dist_max_div', 'atom_1_n_bonds', 'dist_H_0_x', 'tertiary_atom_2', 'cos_f0', 'tertiary_atom_1', 'atom_1_bond_lengths_max', 'dist_O_0_y', 'dist_to_type_std', 'vander_O.y', 'tertiary_angle_2', 'yukawa_H.y', 'dist_C_2_y', 'adC3', 'adC1', 'tertiary_distance_1', 'adC2', 'dist_H_0_y', 'dist_C_1_x', 'molecule_atom_index_1_dist_min_diff', 'dist_H_1_y', 'bond_atom', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'dist_H_2_y', 'dist_N_0_y', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_diff', 'eem_1', 'atom_index_1_cycle_size_mean', 'dist_C_0_y', 'dist_C_2_x', 'dist_xyz', 'dist_H_1_x', 'dist_C_4_x', 'molecule_atom_index_0_dist_std_div', 'dist_C_1_y', 'tertiary_angle_0', 'dist_O_0_x', 'dist_to_type_1_mean', 'inv_distPR', 'atom_1_bond_lengths_min', 'min_molecule_atom_0_dist_xyz', 'dist_C_3_y', 'tertiary_angle_3', 'cos_c1', 'tertiary_distance_4', 'tertiary_atom_3', 'atom_1_bond_lengths_mean', 'dist_C_3_x', 'dist_H_3_y', 'cos_c0_c1', 'cos_c0', 'inv_dist0R', 'max_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
               ]

In [9]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [12]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-21 06:34:08.207422,modeling for 0,68,0.831599,0.941337,0.109738,modeling for 0
1,2019-07-21 06:34:28.080121,modeling for 3,67,0.160207,0.198846,0.038639,modeling for 3
2,2019-07-21 06:34:31.229728,modeling for 1,77,0.277930,0.440667,0.162737,modeling for 1
3,2019-07-21 06:34:38.688148,modeling for 4,67,0.156553,0.242246,0.085693,modeling for 4
4,2019-07-21 06:35:39.057613,modeling for 2,68,0.365101,0.395450,0.030350,modeling for 2
5,2019-07-21 06:36:13.287223,modeling for 6,70,0.191867,0.220793,0.028926,modeling for 6
6,2019-07-21 06:37:54.083516,modeling for 5,79,0.410485,0.437274,0.026789,modeling for 5
7,2019-07-21 06:38:03.106022,modeling for 7,62,0.098279,0.139488,0.041208,modeling for 7


In [13]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.4414013190561284
0 0.9413368511055361
3 0.1988460127465658
1 0.44066665850314923
4 0.24224589451301176
2 0.39545027197188404
6 0.2207930281919972
5 0.43727380870970267
7 0.13948781490764398


In [14]:
df_test_pred.tail()

,index,0,1,2,3,4
2505537,7163684,0.504343,0.249645,0.383658,0.510983,0.713844
2505538,7163685,3.003948,3.678136,3.857650,2.861471,3.893262
2505539,7163686,1.340398,2.126085,2.471289,2.317472,2.880695
2505540,7163687,6.097431,3.617092,2.604197,5.418305,4.289756
2505541,7163688,113.142411,115.394378,114.157580,114.538418,112.525543


In [17]:
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['fc'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_pickle('../../data/oof/lgbm_fc_test.pkl')

In [18]:
df_submit = df_valid_pred.rename(columns={'predict':'fc', 'index':'id'})
df_submit.to_pickle('../../data/oof/lgbm_fc_train.pkl')